# 04.04 generalized minimum residual method (GMRES)

a good choice for large, sparse, nonsymmetric linear systems $Ax = b$.

why does orthogonality matter to a linear system? bc matrices with almost parallel column vectors tend to be ill-conditioned - ie, have large error magnification.

GMRES ~ orthogonal x2: with each iteration,

1. search space basis is reorthogonalized; and
2. backwards error minimized using least squares.

## 1 iterative krylov method

GMRES is a krylov method - which relies on krylov space. krylov space is a vector space spanned by $\{r, Ar, . . . , A^k r\}$, where $r = b - Ax_0$ is the residual vector of the initial guess.

chebyshev minimized runge error by choosing roots; GMRES minimizes error by how it improves the initial guess. (note: not the initial guess itself but the increments to it.)

it looks at the krylov space spanned by $r$ and its products under nonsingular $A$. at each step $k$, the krylov space increases, its basis is reorthogonalized and least squares is applied. (ie, gram-schmidt & householder from previous lecture.)

##### algorithm, GMRES

```
x[0] = initial guess
r = b - A*x[0]
β = ||r|| # L2 norm
TOL = ε[mach]*||A||

q[1] = r/β
for k in range(m) # [0,m) # m = max iterations of krylov subspace
  y = A*q[k]
  for j in range(k) # [0,k)
    h[j,k] = (q[j])ᵀ•y
    y = y - h[j,k]*q[j]

  h[k+1,k] = ||y|| # L2 norm

  if h[k+1,k] > TOL
    q[k+1] = y/h[k+1,k]

  # minimize || H*c[k] - β*e[1]ᵀ ||, where e[1] = [1, 0, ..., 0]
  c[k] = least_squares(H,β*e[1]ᵀ)
  x[k] = Q[k]*c[k] + x[0]

  # check residual for convergence
  r_norm = β * abs(c[k][-1])
  if r_norm <= TOL * β or h[k+1,k] <= TOL:
    break # stop at iteration k

return x[k],k
```

where

*   $x_k$ is $\bar{x}$ for $Ax = b$;

and at iteration $k$,

*   $H$ is the $k+1$ x $k$ matrix minimized with least squares for $c$;
*   $Q_k$ is the $n$ x $k$ matrix of $k$ orthonormal colums; and
*   if $h_{k+1,k} = 0$, $k$ is final step and minimizing $H$ will return the exact solution of $Ax = b$.

arnoldi iteration is gram-schmidt applied to $\{r,Ar,...,A^kr\}$ and results in $AQ_k = Q_{k+1}H_k$. <mark>this is step one of GMRES.</mark> ie,

$$
A\left[\begin{array}{c|c|c}
  \\ \\ q_1 & \dots & q_k \\ \\ \\
\end{array}\right]
=\left[\begin{array}{c|c|c|c}
  \\ \\ q_1 & \dots & q_k & q_{k+1} \\ \\ \\
\end{array}\right]
\left[\begin{array}{c}
  h_{11} & h_{12} & \dots & h_{1k} \\
  h_{21} & h_{22} & \dots & h_{2k} \\
  & h_{32} & \dots & h_{3k} \\
  && \ddots & \vdots \\
  &&& h_{k+1,1k} \\
\end{array}\right].
$$

$A$ is $n$ x $n$, $Q_k$ is $n$ x $k$ and $H_k$ is $(k+1)$ x $k$. usually $k \ll n$, where $n$ is number of unknowns / columns.

$Q_k$, which spans the $k$-dimensional krylov space, is what is searched for the increment $x_{\text{add}}$ that is applied to $x_0$.

\begin{align}
  b - A(x_0 + x_{\text{add}}) &= r - Ax_{\text{add}} \\
  \\
  &\Rightarrow ||Ax_{\text{add}} - r||_2 \\
  \\
  &= ||AQ_kc - r||_2 \\
  \\
  &= ||Q_{k+1}H_kc - r||_2 \\
  &= ||H_kc - Q^T_{k+1}r||_2, \text{ where } Q^T_{k+1}r = \beta\cdot e_1^T = \underbrace{[\; \overbrace{||r||_2}^{\beta}\; 0\; \dots\; 0]^T}_{e_1^T = [1\; 0\; \dots\; 0\; ]^T}.
\end{align}

ie, find $c$ that minimizes the residual above by solving the least squares problem below. <mark>this is step two of GMRES.</mark>

$$
\left[\begin{array}{c}
  h_{11} & h_{12} & \dots & h_{1k} \\
  h_{21} & h_{22} & \dots & h_{2k} \\
  & h_{32} & \dots & h_{3k} \\
  \\
  && \ddots & \vdots \\
  \\
  &&& h_{k+1,k} \\
\end{array}\right]
\left[\begin{array}{c}
  c_1 \\ c_2 \\ \\ \vdots \\ \\ c_k
\end{array}\right]
=
\left[\begin{array}{c}
  ||r||_2 \\ 0 \\ \\ \\ \vdots \\ \\ 0
\end{array}\right]_{k+1 \times 1}
$$

then $x_k = x_0 + x_{\text{add},k}$ where $x_{\text{add},k} = Q_k \times c$.

#### cost, usw

computationally, least squares is what costs the most but usually $k \ll n$, so usually its beneficial. when $h_{k+1,k} = 0$, the problem is zero and the solution $x_k$ exact.

also, bc of least squares, error $||b-Ax_k||_2$ decreases monotonically with $k$.

also, you can do the least squares part once in a while or only at the end if you only need an approximate solution. thats not very math-like but sometimes the real world gets impatient.

also, if a large $n$ runs too long, stop it and restart with $x_0 = x_{\text{stop}}$. and there was much rejoicing.[$^{[1]}$](#ref-01)

## 2 pre-conditioned GMRES

for nonsymmetric $Ax = b$, solve $M^{-1}Ax = M^{-1}b$. ie, residual $r = M^{-1}(b-Ax_0)$ with krylov space iteration $w = M^{-1}Aq_k$.

##### algorithm, GMRES, pre-conditioned

```
x[0] = initial guess
r = inverse(M)*(b - A*x[0])
β = ||r|| # L2 norm
TOL = ε[mach]*||A|| # ok enough for most preconditioned

q[1] = r/β # L2 norm
for k in range(m) # [0,m) # m = max iterations of krylov subspace
  w = inverse(M)*A*q[k]
  for j in range(k) # [0,k)
    h[j,k] = wᵀ•q[j]
    w = w - h[j,k]*q[j]

  h[k+1,k] = ||w|| # L2 norm
  q[k+1] = w/h[k+1,k]

  # minimize || H*c[k] - β*e[1]ᵀ ||, where e[1] = [1, 0, ..., 0]
  c[k] = least_squares(H,β*e[1]ᵀ)
  x[k] = Q[k]*c[k] + x[0]

  # check residual for convergence
  r_norm = β * abs(c[k][-1])
  if r_norm <= TOL * β or h[k+1,k] <= TOL:
    break # stop at iteration k

return x[k],k
```

## resources


<b>krylov subspaces</b>

* excellent summary! [@aerodynamic cfd](https://www.youtube.com/watch?v=B_eSPrYuIuU) (4m)
* rationale, proofs [@daniel an](https://www.youtube.com/watch?v=UgyLaAXqlQ4) (12m)
* textbook-like [@ocw.mit](https://ocw.mit.edu/courses/18-086-mathematical-methods-for-engineers-ii-spring-2006/41750e11021842bc972670e46cc949cc_am64.pdf) (homely but obviously diligent)
* math for the people [@etchzurich](https://people.math.ethz.ch/~mhg/pub/biksm.pdf) (not as homely)
* nerdy reading [@arxiv](https://arxiv.org/abs/1811.09025)

<b>GMRES</b>

* krylov for GMRES [@daniel an](https://www.youtube.com/watch?v=R9DHmkCE9oI&list=PLP1OdTlavJNvV0KDxLjqkmc_RGr5z6WIi&index=4)
* GMRES [@scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.gmres.html)

## references

<a name="ref-01"></a>
1. [and there was much rejoicing...](https://www.youtube.com/watch?v=yciX2meIkXI)